# Steam (Environment - Agent)

In [1]:
# Basic import
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from scipy.stats import invgamma, gamma
from scipy.stats import t as student
import pdb
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, Dot, Concatenate
from tensorflow.keras.models import Model
tf.config.experimental_run_functions_eagerly(True)
print(tf.__version__)

2.0.0


## Environment

In [2]:
class Environment:
    """ 
    Contextual Multi-Armed Bandit environment
    """
    def __init__(self, nb_films, nb_users, 
                 context_size = 2,
                 displayed_users_embedding_size = 2, 
                 displayed_games_embedding_size = 2, 
                 noise_size = 3,
                 seed=None):
        self._nb_games = nb_games
        self._nb_users = nb_users
        self._p = context_size # size of user, size of game
        self._displayed_users_embedding_size = displayed_users_embedding_size
        self._displayed_games_embedding_size = displayed_games_embedding_size
        self._noise_size = noise_size
        self._rng = np.random.RandomState(seed)

    def step(self):
        """ Play an action """
        user = self._rng.randint(0, self._nb_users)
        available_games = np.where(self._available_games[user] == 1)[0]
        return user, available_games
    
    def update(self, user, game):
        reward = self._reward_matrix[user, game]
        self._available_games[user, game] = 0
        return reward
    
    def reset(self):
        #self._games = self._rng.uniform(size=(nb_games, context_size))
        #self._users = self._rng.uniform(size=(nb_users, context_size))
        #-------------------------------------------------------#
        self.user_mean = np.ones(self._p)
        self.user_var = np.ones(self._p)
        self.game_mean = np.ones(self._p)
        self.game_var = np.ones(self._p)
        self._users = self._rng.normal(loc=self.user_mean,
                                                scale=self.user_var,
                                                size=(self._nb_users, self._p))
        self._games = self._rng.normal(loc=self.game_mean,
                                                scale=self.game_var,
                                                size=(self._nb_games, self._p))
        #-------------------------------------------------------#
        self._reward_matrix = np.zeros((nb_users, nb_games))
        for i in range(self._reward_matrix.shape[0]):
            for j in range(self._reward_matrix.shape[1]):
                # reward = np.linalg.norm(self._games[j] - self._users[i], ord=2)
                reward = self._games[j].dot(self._users[i])
                self._reward_matrix[i, j] = reward
        self._reward_matrix = ((self._reward_matrix - np.min(self._reward_matrix)) / np.max(self._reward_matrix) * 4).astype(int) + 1
        self._available_games = np.ones((nb_users, nb_games))
        self._available_films = np.ones((nb_users, nb_games))
        users = deepcopy(self._users)
        return users

    def get_feature_vector(self, user, game):
        user_embedding = self._users[user]
        game_embedding = self._games[game]
        if self._displayed_users_embedding_size + self._displayed_games_embedding_size > 0:
            variables = np.array([user_embedding[:self._displayed_users_embedding_size],
                                  game_embedding[:self._displayed_games_embedding_size]])

            if self._noise_size > 0:
                noise = self._rng.normal(loc=np.ones(self._noise_size),
                                         scale=np.ones(self._noise_size),
                                         size=self._noise_size)
                variables = np.append(variables, noise)
            return variables

## Agent

In [3]:
class RandomAgent:
    """ 
    Random agent
    """
    def __init__(self, seed = None):
        self._rng = np.random.RandomState(seed)
    
    def act(self, available_games):
        action = self._rng.choice(available_games)
        return action

## Experiment

In [4]:
# Basic parameter
nb_users = 30 #number of users in the context
nb_games = 10 #number of games in the context
context_size = 2 #number of different film categories = 2

In [5]:
# Creating the environment
env = Environment(nb_games,nb_users,context_size,2020)
env.reset() #reset and initilize the environment

array([[ 1.48498184,  1.26196845],
       [ 1.93963171,  0.80235644],
       [ 0.71129992,  1.40236516],
       [ 1.2675628 ,  1.50936377],
       [ 1.44465119,  2.40601401],
       [ 0.00538904,  1.07947062],
       [ 1.44188963,  1.6729676 ],
       [ 0.08965077,  0.80236926],
       [ 1.73706095,  2.13451562],
       [ 1.20991524,  1.20128777],
       [ 2.40012652,  0.97874821],
       [ 0.37882481,  0.5025776 ],
       [ 0.60090051,  1.64231227],
       [ 1.57053191, -0.52238166],
       [-1.22937729,  1.41049177],
       [ 1.90333509, -0.01256267],
       [-0.84473841,  1.81383781],
       [ 0.14327327,  2.39216124],
       [ 1.28667635,  1.00436545],
       [-0.2447395 ,  2.36752408],
       [ 0.52334678, -1.09209878],
       [ 2.72619731,  0.35124098],
       [ 0.82016351,  1.84104755],
       [-0.32666989,  0.99943449],
       [ 1.37461262, -0.94690252],
       [ 2.54900226,  0.53197812],
       [ 0.09236578,  0.22169957],
       [-0.08830567,  0.92643129],
       [ 3.53475691,

In [6]:
# Creating the agent
agent = RandomAgent(2020)

We run the experiment and generate some historical data.

In [7]:
# Running several trials
nb_iteration = 100 #how many trials
rating_matrix = np.zeros((env._nb_users, env._nb_games))
users = list()
games = list()
ratings = list()
for i in range(nb_iteration):
    user, available_games = env.step()
    choosen_game = agent.act(available_games)
    reward = env.update(user, choosen_game)
    users.append(user)
    games.append(choosen_game)
    ratings.append(reward)
    rating_matrix[user, choosen_game] = reward
    '''
    print("user = {}, recommended_games = {}, choosen_game = {}".format(user,recommended_games,choosen_game))
    print("reward = {}\n".format(reward))
    '''
    
print("rating matrix: \n", str(rating_matrix))

rating matrix: 
 [[0. 0. 0. 0. 0. 0. 3. 3. 0. 0.]
 [0. 4. 2. 0. 3. 3. 0. 3. 0. 0.]
 [3. 4. 0. 4. 3. 0. 3. 3. 3. 3.]
 [3. 4. 0. 0. 3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 0. 3. 0. 3. 0. 0.]
 [2. 0. 0. 4. 0. 0. 0. 0. 0. 0.]
 [3. 4. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 3. 0. 3. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 3.]
 [2. 4. 3. 0. 3. 3. 3. 0. 4. 3.]
 [2. 0. 2. 5. 3. 0. 3. 0. 0. 0.]
 [0. 0. 0. 3. 3. 0. 3. 0. 3. 2.]
 [3. 0. 3. 0. 0. 0. 0. 3. 0. 0.]
 [2. 0. 0. 0. 0. 0. 0. 2. 4. 0.]
 [0. 0. 4. 3. 0. 0. 3. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 3. 0. 5. 0.]
 [0. 0. 0. 4. 0. 0. 3. 0. 0. 0.]
 [3. 0. 0. 0. 0. 0. 0. 3. 2. 0.]
 [0. 0. 0. 4. 0. 0. 0. 0. 0. 3.]
 [0. 0. 0. 5. 0. 2. 0. 0. 0. 2.]
 [2. 0. 0. 0. 0. 0. 0. 0. 3. 0.]
 [0. 0. 0. 0. 3. 0. 3. 0. 0. 0.]
 [3. 4. 3. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 3. 0. 0. 3. 3. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 4. 0.]
 [0. 0. 0. 0. 0. 0. 3. 3. 0. 3.]
 [0. 0. 3. 3. 0. 0. 2. 0. 0. 2.]
 [0. 3. 0. 3. 3. 2. 0. 0. 0. 2.]
 [2. 5. 0. 0. 0. 0. 4. 3. 0. 4.]
 [2. 0. 0. 0. 0. 0. 0. 0. 

## Regression model

In [8]:
class RegressionModel(Model):
    def __init__(self, embedding_size, max_user, max_game):
        super().__init__()
        
        self.user_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_user,
                                        input_length=1,
                                        name='user_embedding')
        self.game_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_game,
                                        input_length=1,
                                        name='game_embedding')
        
        # The following two layers don't have parameters.
        self.flatten = Flatten()
        self.dot = Dot(axes=1)
        
    def call(self, inputs):
        user_inputs = inputs[0]
        game_inputs = inputs[1]
        
        user_vecs = self.flatten(self.user_embedding(user_inputs))
        game_vecs = self.flatten(self.game_embedding(game_inputs))
        
        y = self.dot([user_vecs, game_vecs])
        return y

model = RegressionModel(64, nb_users, nb_games)
model.compile(optimizer="adam", loss='mae')

In [9]:
users = np.array(users)
games = np.array(games)
ratings = np.array(ratings)

history = model.fit([users, games], ratings,
                    batch_size=64, epochs=100, validation_split=0.1,
                    shuffle=True)

embeddings = model.get_weights()

Train on 90 samples, validate on 10 samples
Epoch 1/100
90/90 [==============================] - 0s 521us/sample - loss: 3.0442 - val_loss: 3.1995
Epoch 2/100
90/90 [==============================] - 0s 473us/sample - loss: 3.0419 - val_loss: 3.1993
Epoch 3/100
90/90 [==============================] - 0s 458us/sample - loss: 3.0399 - val_loss: 3.1992
Epoch 4/100
90/90 [==============================] - 0s 333us/sample - loss: 3.0378 - val_loss: 3.1990
Epoch 5/100
90/90 [==============================] - 0s 422us/sample - loss: 3.0358 - val_loss: 3.1988
Epoch 6/100
90/90 [==============================] - 0s 418us/sample - loss: 3.0337 - val_loss: 3.1985
Epoch 7/100
90/90 [==============================] - 0s 425us/sample - loss: 3.0315 - val_loss: 3.1982
Epoch 8/100
90/90 [==============================] - 0s 403us/sample - loss: 3.0292 - val_loss: 3.1978
Epoch 9/100
90/90 [==============================] - 0s 352us/sample - loss: 3.0268 - val_loss: 3.1973
Epoch 10/100
90/90 [=========

90/90 [==============================] - 0s 394us/sample - loss: 1.3695 - val_loss: 2.0745
Epoch 80/100
90/90 [==============================] - 0s 404us/sample - loss: 1.3172 - val_loss: 2.0343
Epoch 81/100
90/90 [==============================] - 0s 399us/sample - loss: 1.2632 - val_loss: 1.9934
Epoch 82/100
90/90 [==============================] - 0s 383us/sample - loss: 1.2109 - val_loss: 1.9521
Epoch 83/100
90/90 [==============================] - 0s 388us/sample - loss: 1.1614 - val_loss: 1.9099
Epoch 84/100
90/90 [==============================] - 0s 348us/sample - loss: 1.1136 - val_loss: 1.8675
Epoch 85/100
90/90 [==============================] - 0s 344us/sample - loss: 1.0710 - val_loss: 1.8259
Epoch 86/100
90/90 [==============================] - 0s 367us/sample - loss: 1.0304 - val_loss: 1.7849
Epoch 87/100
90/90 [==============================] - 0s 514us/sample - loss: 0.9928 - val_loss: 1.7441
Epoch 88/100
90/90 [==============================] - 0s 488us/sample - loss:

## Embedding Agent

In [10]:
def cosine(x, y):
    dot = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    cos = dot / (norm_x * norm_y)
    return cos

In [11]:
class EmbeddingAgent:
    """ 
    Embedding Agent
    """
    def __init__(self, user_embeddings, game_embeddings):
        self._game_embeddings = game_embeddings
        self._user_embeddings = user_embeddings
    
    def act(self, user, available_games):
        user_embedding = self._user_embeddings[user]
        dot_products = self._game_embeddings @ user_embedding
        user_embedding_norm = np.linalg.norm(user_embedding)
        all_item_norms = np.linalg.norm(self._game_embeddings, axis=1)
        norm_products = user_embedding_norm * all_item_norms
        sims = dot_products / (norm_products)
        sims = np.argsort(sims)[::-1]
        mask = np.in1d(sims, available_games)
        sims = sims[mask]
        return sims[0]

## Trying the embedding agent

In [12]:
# Creating the agent
agent = EmbeddingAgent(embeddings[0], embeddings[1])

# Running several trials
nb_iteration = 20 #how many trials
for i in range(nb_iteration):
    user, available_games = env.step()
    choosen_game = agent.act(user, available_games)
    reward = env.update(user, choosen_game)
    rating_matrix[user, choosen_game] = reward
    print("user = {}, available games = {}, choosen_game = {}".format(user,available_games,choosen_game))
    print("reward = {}\n".format(reward))

user = 9, available games = [3 7], choosen_game = 7
reward = 3

user = 23, available games = [0 1 2 4 5 8 9], choosen_game = 9
reward = 2

user = 7, available games = [0 1 3 6 7 8 9], choosen_game = 1
reward = 3

user = 13, available games = [1 2 3 4 5 6 9], choosen_game = 1
reward = 3

user = 4, available games = [0 1 2 4 6 8 9], choosen_game = 9
reward = 3

user = 12, available games = [1 3 4 5 6 8 9], choosen_game = 8
reward = 3

user = 10, available games = [1 5 7 8 9], choosen_game = 9
reward = 3

user = 4, available games = [0 1 2 4 6 8], choosen_game = 4
reward = 3

user = 23, available games = [0 1 2 4 5 8], choosen_game = 4
reward = 2

user = 16, available games = [0 1 2 4 5 7 8 9], choosen_game = 9
reward = 2

user = 26, available games = [0 1 4 5 7 8], choosen_game = 4
reward = 2

user = 5, available games = [1 2 4 5 6 7 8 9], choosen_game = 7
reward = 3

user = 26, available games = [0 1 5 7 8], choosen_game = 7
reward = 2

user = 26, available games = [0 1 5 8], choosen_ga

## Deep regression model

In [10]:
class DeepRegressionModel(Model):

    def __init__(self, embedding_size, max_user, max_game):
        super().__init__()
        
        self.user_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_user,
                                        input_length=1,
                                        name='user_embedding')
        self.game_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_game,
                                        input_length=1,
                                        name='game_embedding')
        
        self.flatten = Flatten()
        self.concat = Concatenate()
        
        self.dense1 = Dense(16, activation="relu")
        self.dense2 = Dense(8, activation="relu")
        
    def call(self, inputs, training=False):
        user_inputs = inputs[0]
        game_inputs = inputs[1]
        feature_inputs = inputs[2]
        print(feature_inputs)

        
        user_vecs = self.flatten(self.user_embedding(user_inputs))
        game_vecs = self.flatten(self.game_embedding(game_inputs))
        
        # input_vecs = self.concat([user_vecs, game_vecs, self.flatten(feature_inputs)])
        input_vecs = self.concat([user_vecs, game_vecs])
        
        y = self.dense1(input_vecs)
        y = self.dense2(y)
        
        return y
        
model = DeepRegressionModel(64, nb_users, nb_games)
model.compile(optimizer="adam", loss='mae')

In [11]:
users = np.array(users)
games = np.array(games)
ratings = np.array(ratings)

features = []
for i in range(len(users)):
    features.append(env.get_feature_vector(users[i], games[i]))
features = np.float64(features)

history = model.fit([users, games, features], ratings,
                    batch_size=64, epochs=100, validation_split=0.1,
                    shuffle=True)

embeddings = model.get_weights()

Tensor("input_3:0", shape=(None, 7), dtype=float32)
Train on 90 samples, validate on 10 samples
Epoch 1/100
Tensor("Cast:0", shape=(None, 7), dtype=float32)
Tensor("Cast:0", shape=(None, 7), dtype=float32)
90/90 [==============================] - 1s 6ms/sample - loss: 2.2155 - val_loss: 1.6526
Epoch 2/100
90/90 [==============================] - 0s 115us/sample - loss: 2.1973 - val_loss: 1.6333
Epoch 3/100
90/90 [==============================] - 0s 180us/sample - loss: 2.1791 - val_loss: 1.6140
Epoch 4/100
90/90 [==============================] - 0s 202us/sample - loss: 2.1615 - val_loss: 1.5945
Epoch 5/100
90/90 [==============================] - 0s 213us/sample - loss: 2.1436 - val_loss: 1.5754
Epoch 6/100
90/90 [==============================] - 0s 193us/sample - loss: 2.1256 - val_loss: 1.5560
Epoch 7/100
90/90 [==============================] - 0s 204us/sample - loss: 2.1078 - val_loss: 1.5365
Epoch 8/100
90/90 [==============================] - 0s 203us/sample - loss: 2.0895 - v

Epoch 77/100
90/90 [==============================] - 0s 174us/sample - loss: 1.2286 - val_loss: 0.7050
Epoch 78/100
90/90 [==============================] - 0s 176us/sample - loss: 1.2284 - val_loss: 0.7049
Epoch 79/100
90/90 [==============================] - 0s 157us/sample - loss: 1.2282 - val_loss: 0.7047
Epoch 80/100
90/90 [==============================] - 0s 165us/sample - loss: 1.2280 - val_loss: 0.7046
Epoch 81/100
90/90 [==============================] - 0s 170us/sample - loss: 1.2278 - val_loss: 0.7044
Epoch 82/100
90/90 [==============================] - 0s 166us/sample - loss: 1.2276 - val_loss: 0.7043
Epoch 83/100
90/90 [==============================] - 0s 167us/sample - loss: 1.2275 - val_loss: 0.7042
Epoch 84/100
90/90 [==============================] - 0s 171us/sample - loss: 1.2273 - val_loss: 0.7041
Epoch 85/100
90/90 [==============================] - 0s 171us/sample - loss: 1.2272 - val_loss: 0.7040
Epoch 86/100
90/90 [==============================] - 0s 179us/s

In [12]:
# Creating the agent
agent = EmbeddingAgent(embeddings[0], embeddings[1])

# Running several trials
nb_iteration = 20 #how many trials
for i in range(nb_iteration):
    user, available_games = env.step()
    choosen_game = agent.act(user, available_games)
    reward = env.update(user, choosen_game)
    rating_matrix[user, choosen_game] = reward
    print("user = {}, available games = {}, choosen_game = {}".format(user,available_games,choosen_game))
    print("reward = {}\n".format(reward))

user = 25, available games = [0 1 2 5 7 8 9], choosen_game = 2
reward = 1

user = 19, available games = [0 1 2 3 4 5 6 7 8 9], choosen_game = 7
reward = 2

user = 10, available games = [2 4 7 8], choosen_game = 2
reward = 1

user = 13, available games = [0 1 2 3 4 5 7 8 9], choosen_game = 9
reward = 3

user = 1, available games = [4 5 6 8], choosen_game = 8
reward = 2

user = 18, available games = [1 2 3 4 5 6 7 8 9], choosen_game = 2
reward = 3

user = 19, available games = [0 1 2 3 4 5 6 8 9], choosen_game = 8
reward = 2

user = 19, available games = [0 1 2 3 4 5 6 9], choosen_game = 4
reward = 1

user = 2, available games = [2 4 8 9], choosen_game = 9
reward = 2

user = 8, available games = [0 1 4 7 8 9], choosen_game = 9
reward = 2

user = 6, available games = [0 1 2 3 4 6 9], choosen_game = 9
reward = 4

user = 4, available games = [0 4 8 9], choosen_game = 9
reward = 2

user = 10, available games = [4 7 8], choosen_game = 8
reward = 1

user = 15, available games = [0 2 4 7 9], ch